In [8]:
from download_hvpy_wdates import calculate_timestamps_for_flare, FlareEvent, extract_datetime
from datetime import datetime, timedelta, timezone
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from hvpy import takeScreenshot, create_layers
from typing import List, Tuple

data_dir = '../../data/'

In [4]:
flares_path = os.path.join(data_dir, 'flares_GOES.csv')

df = pd.read_csv(flares_path)
print(len(df))
df.head()

7093


,kb_archivid,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,event_coord1,event_coord2,event_coord3
0,ivo://helio-informatics.org/FL_SECstandard_201...,2011-10-01 03:50:00.000,2011-10-01 03:56:00.000,2011-10-01 04:04:00.000,C1.3,11305,0,0,NaN
1,ivo://helio-informatics.org/FL_SECstandard_201...,2011-10-01 04:44:00.000,2011-10-01 04:50:00.000,2011-10-01 04:53:00.000,C4.1,11302,0,0,NaN
2,ivo://helio-informatics.org/FL_SECstandard_201...,2011-10-01 08:56:00.000,2011-10-01 09:59:00.000,2011-10-01 10:17:00.000,M1.2,11305,6,10,NaN
3,ivo://helio-informatics.org/FL_SECstandard_201...,2011-10-02 00:37:00.000,2011-10-02 00:50:00.000,2011-10-02 00:59:00.000,M3.9,11305,12,9,NaN
4,ivo://helio-informatics.org/FL_SECstandard_201...,2011-10-02 03:10:00.000,2011-10-02 03:49:00.000,2011-10-02 04:13:00.000,C1.5,11302,0,0,NaN


In [5]:
from dataclasses import dataclass


@dataclass
class FlareEvent:
    archive_id: str
    start_time: datetime
    peak_time: datetime
    end_time: datetime
    flare_class: str

def extract_datetime(input) -> datetime:
    return datetime.strptime(input, "%Y-%m-%d %H:%M:%S.%f")

flare_events: List[FlareEvent] = []
for index, row in df.iterrows():
    # Access row values using row['column_name']
    
    flare_event = FlareEvent(archive_id=row['kb_archivid'], 
                             start_time=extract_datetime(row['event_starttime']), 
                             peak_time=extract_datetime(row['event_peaktime']), 
                             end_time=extract_datetime(row['event_endtime']), 
                             flare_class=row['fl_goescls'])
    flare_events.append(flare_event)

print(len(flare_events))

7093


In [5]:

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
from datetime import datetime, timedelta
source_id = 19

valid_flares: List[Tuple[FlareEvent, List[datetime]]] = []

# for flare_event in tqdm(flare_events):
#     valid, dates = calculate_timestamps_for_flare(flare_event, source_id=source_id)
#     if valid:
#         valid_flares.append((flare_event, dates))

# Define a function to process each flare event
def process_flare(flare_event):
    valid, dates = calculate_timestamps_for_flare(flare_event, source_id=source_id, delta_before_start=timedelta(hours=10))
    if valid:
        return (flare_event, dates)
    else:
        return None

# Create a ThreadPoolExecutor with the desired number of threads
num_threads = 2  # Adjust the number of threads as needed
with ThreadPoolExecutor() as executor:
    print(executor._max_workers)
    # Use the executor to process each flare event in parallel
    results = list(tqdm(executor.map(process_flare, flare_events), total=len(flare_events)))

# Filter out None values and store the valid flares
valid_flares = [result for result in results if result is not None]


20


100%|██████████| 7093/7093 [19:05<00:00,  6.19it/s]


In [1]:
import pickle
import os

pickle_file = 'valid_flares_10h.pickle'

if not os.path.exists(pickle_file):
    # Save the data to a pickle file
    with open(pickle_file, 'wb') as f:
        pickle.dump(valid_flares, f)
else:
    print('Loading from pickle')


Loading from pickle


In [5]:
import pickle
from dataclasses import dataclass
from datetime import datetime


@dataclass
class FlareEvent:
    archive_id: str
    start_time: datetime
    peak_time: datetime
    end_time: datetime
    flare_class: str

# Load the data from the pickle file
pickle_file = 'valid_flares_10h.pickle'
with open(pickle_file, 'rb') as f:
    valid_flares = pickle.load(f)

# Print the loaded data
print(len(valid_flares))


6171


In [6]:
valid_flares[0]

(FlareEvent(archive_id='ivo://helio-informatics.org/FL_SECstandard_20120111_192531_20111001035000', start_time=datetime.datetime(2011, 10, 1, 3, 50), peak_time=datetime.datetime(2011, 10, 1, 3, 56), end_time=datetime.datetime(2011, 10, 1, 4, 4), flare_class='C1.3'),
 [datetime.datetime(2011, 9, 30, 17, 50, 4),
  datetime.datetime(2011, 9, 30, 19, 51, 34),
  datetime.datetime(2011, 9, 30, 21, 52, 19),
  datetime.datetime(2011, 9, 30, 23, 53, 49),
  datetime.datetime(2011, 10, 1, 1, 54, 34),
  datetime.datetime(2011, 10, 1, 3, 56, 4)])

In [9]:

file_path = flares_path = os.path.join(data_dir, 'flare_data.pkl')
flare_df = pd.read_pickle(file_path)
flare_df.head()

mapped_list = [FlareEvent(archive_id=index, 
                          start_time=row['beginTime'].to_pydatetime(), 
                          peak_time=row['peakTime'].to_pydatetime(), 
                          end_time=row['endTime'].to_pydatetime(), 
                          flare_class=row['classType'])
               for index, row in flare_df.iterrows()]


filtered = [i for i in mapped_list if i.flare_class.startswith('M')]
print(len(filtered), len(mapped_list))

440 562


## Valid flares

The `valid_flares.pickle` file contains all flares with sufficient images & spread between start and finish. Next step is to download the data.

In [13]:
from tqdm import tqdm
from hvpy import createScreenshot, takeScreenshot, create_layers, DataSource
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import List, Tuple
from datetime import datetime, timedelta

source_id = 19
w = 1000

def download_flare(flare_event: FlareEvent, date: datetime, w: int):
    fid = flare_event.archive_id.replace('ivo://helio-informatics.org/', '')
    date_str = date.replace(tzinfo=timezone.utc).strftime("%Y-%m-%dT%H_%M_%SZ")
    folder_path = os.path.join(data_dir, 'flare_images', fid, 'source_19')
    filename = os.path.join(folder_path, f"{date_str}")
    os.makedirs(folder_path, exist_ok=True)

    if not os.path.exists(f"{filename}.png"):
        # print(f"Downloading {filename}")
        createScreenshot(
            date=date,
            layers=create_layers([(DataSource.HMI_MAG, 100)]),
            eventLabels=False,
            watermark=False,
            imageScale=1,
            x1=-1 * w,
            y1=-1 * w,
            x2=w,
            y2=w,
            filename=filename,
            overwrite=False
        )
    else:
        # print(f"Skipping {filename}")
        pass
            
            
def download_flares_parallel(flare_list: List[Tuple[FlareEvent, List[datetime]]], w: int):
    with ThreadPoolExecutor() as executor:
        futures = []
        for flare_event, dates in flare_list:
            for date in dates:
                future = executor.submit(download_flare, flare_event, date, w)
                futures.append(future)

        # Wait for all tasks to complete
        for future in tqdm(futures):
            future.result()

download_flares_parallel(valid_flares, w)


  0%|          | 59/37026 [00:20<3:29:33,  2.94it/s]


In [12]:
import pandas as pd

p = os.path.join(data_dir, 'flare_data.pkl')
p2 = os.path.join(data_dir, 'valid_flares_10h.pickle')

mylist = pickle.load(open(p2, 'rb'))
df = pd.read_pickle(p)
# df.head()
print(mylist[0][0])

FlareEvent(archive_id='ivo://helio-informatics.org/FL_SECstandard_20120111_192531_20111001035000', start_time=datetime.datetime(2011, 10, 1, 3, 50), peak_time=datetime.datetime(2011, 10, 1, 3, 56), end_time=datetime.datetime(2011, 10, 1, 4, 4), flare_class='C1.3')
